In [ ]:
import altair as alt
import pandas as pd
import json

alt.data_transformers.disable_max_rows()

# --------------------------------
# 1. Read New York's GeoJSON & Airbnb Dataset 
# --------------------------------
with open('new-york-city-boroughs.geojson', 'r') as f:
    nyc_boroughs = json.load(f)

df = pd.read_csv("listings (1).csv")
columns_needed = [
    'id', 'name', 'host_id', 'neighbourhood_group', 'neighbourhood', 'room_type',
    'latitude', 'longitude', 'price', 'availability_365', 'last_review', 'number_of_reviews', 'host_name'
]
df = df[columns_needed].dropna(subset=['latitude', 'longitude', 'price'])
df['price'] = pd.to_numeric(df['price'], errors='coerce')
df = df.dropna(subset=['price'])

df['last_review'] = pd.to_datetime(df['last_review'], errors='coerce')
df['year'] = df['last_review'].dt.year

top_hosts = df.groupby(['host_id', 'host_name']).size().reset_index(name='listing_count')
df = df.merge(top_hosts, on=['host_id', 'host_name'], how='left')  
df['order_value'] = df['listing_count'].fillna(0)


# --------------------------------
# 2. Filter Settings
# --------------------------------

def get_quarter_filter():
    return (
        "(quarterSelect == 'ALL') ||"  
        "(quarterSelect == 'Q1 (Jan-Mar)' && month(datum.last_review) >= 1 && month(datum.last_review) <= 3) ||"
        "(quarterSelect == 'Q2 (Apr-Jun)' && month(datum.last_review) >= 4 && month(datum.last_review) <= 6) ||"
        "(quarterSelect == 'Q3 (Jul-Sep)' && month(datum.last_review) >= 7 && month(datum.last_review) <= 9) ||"
        "(quarterSelect == 'Q4 (Oct-Dec)' && month(datum.last_review) >= 10 && month(datum.last_review) <= 12)"
    )


review_slider = alt.binding_range(
    name="Min Reviews: ",
    min=0,
    max=int(df['number_of_reviews'].max()), 
    step=5
)
review_select = alt.param(
    name="reviewThreshold",
    value=0,  
    bind=review_slider
)

min_highlight_slider = alt.binding_range(
    name='Min Highlight Price: ',
    min=int(df['price'].min()),
    max=int(df['price'].max()),
    step=5
)
min_highlight_select = alt.param(
    name='minHighlightPrice',
    value=0,  
    bind=min_highlight_slider
)

max_highlight_slider = alt.binding_range(
    name='Max Highlight Price: ',
    min=int(df['price'].min()),
    max=int(df['price'].max()),
    step=5
)
max_highlight_select = alt.param(
    name='maxHighlightPrice',
    value=0, 
    bind=max_highlight_slider
)


host_selection = alt.selection_point(
    fields=['host_id'], 
    empty='none',  
    name="selectedHost",
    toggle=True  
)

year_options = sorted(df['year'].dropna().unique().astype(int))
year_binding = alt.binding_select(options=year_options, name="Select Year: ")
year_select = alt.param(name="yearSelect", value=2024, bind=year_binding)

quarter_options = ["ALL", "Q1 (Jan-Mar)", "Q2 (Apr-Jun)", "Q3 (Jul-Sep)", "Q4 (Oct-Dec)"]
quarter_binding = alt.binding_select(options=quarter_options, name="Select Quarter: ")
quarter_select = alt.param(name="quarterSelect", value="ALL", bind=quarter_binding)

top_region_df = df.groupby(['year', 'neighbourhood_group'])['price'].mean().reset_index()
top_region_df = top_region_df[top_region_df['year'] == 2024]  
top_region = top_region_df.loc[top_region_df['price'].idxmax(), 'neighbourhood_group']

region_options = ["All"] + sorted(list(df['neighbourhood_group'].unique()))
region_binding = alt.binding_select(options=region_options, name="Select Region: ")
region_select = alt.param(name="regionSelect", value="All", bind=region_binding)

room_options = ["All"] + list(df['room_type'].unique())
room_binding = alt.binding_select(options=room_options, name="Room Type: ")
room_select = alt.param(name="roomSelect", value="All", bind=room_binding)

# --------------------------------
# 3. Task1
# Description: 
# Find the neighbourhood_group with the highest average housing price in 2024 (from January 1, 2024, to December 31, 2024).
# --------------------------------

borough_color_scale = alt.Scale(
    domain=['Bronx','Brooklyn','Manhattan','Queens','Staten Island'],
    range=['#fb9a99','#a6cee3','#1f78b4','#b2df8a','#fdbf6f']
)

avg_price_chart = alt.Chart(df).transform_filter(
    'datum.year == yearSelect'
).transform_filter(
    get_quarter_filter()
).transform_filter(
    'roomSelect == "All" || datum.room_type == roomSelect'
).transform_filter(
    'datum.number_of_reviews >= reviewThreshold'
).mark_bar().encode(
    x=alt.X('neighbourhood_group:N', title="Borough"),
    y=alt.Y('mean(price):Q', title="Avg Price"),
    color=alt.Color(
        'neighbourhood_group:N', 
        scale=borough_color_scale,),  
    tooltip=['neighbourhood_group', 'mean(price)']
).properties(width=600, height=300, title="Average Price per Borough"
).add_params(year_select, quarter_select, room_select, review_select)


boxplot = alt.Chart(df).transform_filter(
    'datum.year == yearSelect'
).transform_filter(
    get_quarter_filter()
).transform_filter(
    'roomSelect == "All" || datum.room_type == roomSelect'
).transform_filter(
    'datum.number_of_reviews >= reviewThreshold'
).mark_boxplot().encode(
    x=alt.X('neighbourhood_group:N', title="Borough"),
    y=alt.Y('price:Q', title="Price"),
    color=alt.Color('neighbourhood_group:N')
).properties(width=600, height=300, title="Price Distribution by Borough"
).add_params(year_select, quarter_select, room_select, review_select)

# --------------------------------
# 4. Task2
# Description: 
# Find the proportion of “Private room” listings 
# priced between 0-200 in the highest-priced neighbourhood_group of 2024.
# --------------------------------
hist = alt.Chart(df).transform_filter(
    'datum.year == yearSelect'
).transform_filter(
    get_quarter_filter()
).transform_filter(
    'regionSelect == "All" || datum.neighbourhood_group == regionSelect'
).transform_filter(
    'roomSelect == "All" || datum.room_type == roomSelect'
).transform_filter(
    'datum.number_of_reviews >= reviewThreshold'
).mark_bar().encode(
    x=alt.X('price:Q', bin=alt.Bin(step=200), title="Price Range"),
    y=alt.Y('count()', title="Number of Listings"),
    color=alt.Color('neighbourhood_group:N', 
                    scale=borough_color_scale),
    tooltip=[  
        alt.Tooltip('price:Q', title='Price Range'),
        alt.Tooltip('count()', title='Listings Count'),
        alt.Tooltip('room_type:N', title='Room Type'),
        alt.Tooltip('neighbourhood_group:N', title='Borough')
    ]
).properties(width=600, height=300, title="Price Distribution (Filtered)"
).add_params(
    year_select, quarter_select, room_select, review_select, region_select
)


#  --------------------------------
# 6. Task3
# Description: 
# Identify the top three host names based on the number of their listings 
# that received more than 50 reviews during Q4 of 2024 (October 1 - December 31).
# --------------------------------
host_chart = alt.Chart(df).transform_filter(
    'datum.year == yearSelect'
).transform_filter(
    get_quarter_filter()
).transform_filter(
    'datum.number_of_reviews >= reviewThreshold'
).transform_filter(
    'roomSelect == "All" || datum.room_type == roomSelect'
).transform_filter(
    'regionSelect == "All" || datum.neighbourhood_group == regionSelect'
).transform_aggregate(
    listing_count='count()',
    groupby=['host_id', 'host_name']  
).transform_lookup(
    lookup='host_id',
    from_=alt.LookupData(df[['host_id', 'host_name']].drop_duplicates(), 'host_id', ['host_name'])
).transform_calculate(  
    host_label="datum.host_name + ' (' + datum.host_id + ')'"
).transform_window(
    rank='rank()', sort=[alt.SortField('listing_count', order='descending')]
).transform_filter(
    'datum.rank <= 20'
).mark_bar().encode(
    x=alt.X('listing_count:Q', title='Number of Listings'),
    y=alt.Y('host_label:N', sort='-x', title='Host Name'),  
    color=alt.condition(host_selection, alt.value("red"), alt.value("steelblue")),
    tooltip=['host_name', 'host_id', 'listing_count']
).properties(
    width=600,
    height=400,
    title='Top 20 Active Hosts by Listing Count'
).add_params(
    year_select, quarter_select, room_select, review_select, region_select, host_selection
)


#  --------------------------------
# 5. Task4
# Description: 
# Display the geographical distribution of listings for the host with the highest number of listings identified in Task 3.
# --------------------------------
borough_map = alt.Chart(
    alt.Data(values=nyc_boroughs['features'])
).mark_geoshape(
    stroke='white'
).encode(
    color=alt.Color('properties.name:N', scale=borough_color_scale, legend=alt.Legend(title="Borough"))
).project('mercator').properties(
    width=600,
    height=800,
    title="NYC Boroughs"
)


base_points = alt.Chart(df).mark_circle(size=30).encode(
    longitude='longitude:Q',
    latitude='latitude:Q',
    color=alt.Color('price:Q', scale=alt.Scale(scheme='viridis'), title='Price'),
    size=alt.value(30),  
    opacity=alt.value(0.5),  
    order=alt.Order('order_value:Q', sort="descending"),  
    tooltip=['id', 'name', 'price', 'neighbourhood', 'room_type', 'host_name']
).transform_filter(
    'datum.year == yearSelect'
).transform_filter(
    get_quarter_filter()
).transform_filter(
    'roomSelect == "All" || datum.room_type == roomSelect'
).transform_filter(
    'regionSelect == "All" || datum.neighbourhood_group == regionSelect'
).transform_filter(
    'datum.number_of_reviews >= reviewThreshold'
)

highlighted_points = alt.Chart(df).mark_circle(size=35, color='red').encode(
    longitude='longitude:Q',
    latitude='latitude:Q',
    tooltip=['id', 'name', 'price', 'neighbourhood', 'room_type', 'host_name']
).transform_filter(
    host_selection  
)

final_map = borough_map + base_points + highlighted_points  #


# --------------------------------
# 7. Concat all charts
# --------------------------------
final_chart = alt.vconcat(
    alt.hconcat(boxplot,avg_price_chart),
    alt.hconcat(alt.vconcat(hist,host_chart) ,final_map),
)

final_chart


In [2]:
import altair as alt
import re

final_chart.save('price_distribution.html')

with open('price_distribution.html', 'r', encoding='utf-8') as f:
    html_content = f.read()

tasks_content = '''
    <h3>Tasks</h3>
    <ol>
    <li><b>Find the neighbourhood_group with the highest average housing price in 2024 (from January 1, 2024, to December 31, 2024).</b><br></li>

    <li><b>Find the proportion of “Private room” listings priced between 0-200 in the highest-priced neighbourhood_group of 2024.</b><br>
    </li>

    <li><b>Identify the top three host names based on the number of their listings that received more than 50 reviews during Q4 of 2024.</b><br></li>

    <li><b>Display the geographical distribution of listings for the host with the highest number of listings identified in Task 3.</b><br></li>
    </ol>
'''

modified_html = re.sub(
    r'<body>',
    f'<body>{tasks_content}',
    html_content
)

# 将修改后的 HTML 文件写入
with open('system_A.html', 'w', encoding='utf-8') as f:
    f.write(modified_html)

print("Tasks added to system_A.html successfully!")

NameError: name 'final_chart' is not defined